In [0]:
# DBTITLE 0,0. Librerías necesarias
import pandas as pd
import json
import uuid
from datetime import datetime
from pyspark.sql.functions import col, lit, current_timestamp, udf, split
from pyspark.sql.types import StringType, BooleanType, ArrayType
from delta.tables import DeltaTable

# DBTITLE 1,1. Widgets y constantes
dbutils.widgets.text("catalog_name", "workspace", "Catálogo de UC donde residen las tablas")
dbutils.widgets.text("schema_name", "dq_framework", "Esquema de UC donde residen las tablas")
dbutils.widgets.text("tables_sheet", "Tablas", "Hoja Excel: Tablas")
dbutils.widgets.text("rules_sheet", "Reglas", "Hoja Excel: Reglas")
dbutils.widgets.text("validations_sheet", "Validaciones", "Hoja Excel: Validaciones")
dbutils.widgets.text("excel_file_path", "/Volumes/workspace/dq_framework/template/configValidaciones.xlsx", "Ruta del Excel")

CATALOG = dbutils.widgets.get("catalog_name")
SCHEMA = dbutils.widgets.get("schema_name")
CONFIG_SHEET = dbutils.widgets.get("tables_sheet")
LIBRARY_SHEET = dbutils.widgets.get("rules_sheet")
VALIDATIONS_SHEET = dbutils.widgets.get("validations_sheet")
EXCEL_PATH = dbutils.widgets.get("excel_file_path")

TABLE_CONFIG = f"{CATALOG}.{SCHEMA}.dq_tables_config"
RULE_LIB_TABLE = f"{CATALOG}.{SCHEMA}.dq_rules_library"
CATALOG_TABLE = f"{CATALOG}.{SCHEMA}.dq_validations_catalog"

# Columnas esperadas por hoja
CONFIG_EXPECTED_COLS = ['Id_tabla', 'Nombre_tabla', 'Nombre_técnico', 'Clave_primaria', 'Tabla_staging_evidencias']
LIBRARY_EXPECTED_COLS = ['Id_regla', 'Nombre_técnico', 'Nombre_funcional', 'Descripción', 
                         'Dimensión_reglas', 'Tipo_implementación', 'Etiquetas', 'Clase', 
                         'Propietario', 'Fecha_actualizacion', 'Actualizado_por']
VALIDATIONS_EXPECTED_COLS = ['Id_validación','Id_regla','Id_tabla','Nombre_técnico','Dominio_tabla',
                             'Definición_perimetro','Param_columnas','Param_valor','Param_valor_min',
                             'Param_valor_max','Param_conjunto','Param_tipo_dato','Param_expresion',
                             'Param_query_SQL','Param_merge_columnas','Severidad','Owner','Etiquetas',
                             'Actualizado_por','Fecha_actualización','Validación_activa']

# DBTITLE 2,2. Leer Excel y validar columnas
def load_excel_sheet(excel_path, sheet_name, expected_cols):
    errors_list = []
    try:
        df_pd = pd.read_excel(excel_path, sheet_name=sheet_name, dtype=str).fillna('')
        missing_cols = [c for c in expected_cols if c not in df_pd.columns]
        if missing_cols:
            raise ValueError(f"Faltan columnas obligatorias: {missing_cols}")
        df_pd = df_pd.dropna(how='all')
        if df_pd.empty:
            print(f"Hoja '{sheet_name}' vacía. No se sincroniza.")
            return None, errors_list
        return spark.createDataFrame(df_pd), errors_list
    except Exception as e:
        errors_list.append({"sheet": sheet_name, "fila": None, "error": str(e)})
        return None, errors_list

# DBTITLE 3,3. Construir JSON de validaciones
def build_json_definition(technical_name, rule_type, param_columnas_str, param_valor, param_min, param_max, param_conjunto_str,
                          param_sql, param_tipo, param_query, param_merge):
    params_dict = {}
    try:
        if rule_type == 'BUILT-IN':
            if technical_name in ["is_not_null","is_not_empty","is_not_null_and_not_empty","is_not_in_future"]:
                cols = [c.strip() for c in param_columnas_str.split(',') if c]
                if cols: params_dict["column"] = cols[0]
            elif technical_name == "is_unique":
                cols = [c.strip() for c in param_columnas_str.split(',') if c]
                if cols: params_dict["columns"] = cols
            elif technical_name in ["is_in_list","is_not_null_and_is_in_list"]:
                col_name = param_columnas_str.strip() if param_columnas_str else None
                if col_name: params_dict["column"] = col_name
                allowed_values = [v.strip() for v in param_conjunto_str.split(',') if v]
                if allowed_values: params_dict["allowed"] = allowed_values
            elif technical_name == "regex_match":
                col_name = param_columnas_str.strip() if param_columnas_str else None
                if col_name: params_dict["column"] = col_name
                if param_valor: params_dict["regex"] = param_valor
            elif technical_name in ["is_equal_to","is_not_equal_to"]:
                col_name = param_columnas_str.strip() if param_columnas_str else None
                if col_name: params_dict["column"] = col_name
                if param_valor: params_dict["value"] = param_valor
            elif technical_name == "is_valid_date":
                col_name = param_columnas_str.strip() if param_columnas_str else None
                if col_name: params_dict["column"] = col_name
                if param_valor: params_dict["date_format"] = param_valor
            elif technical_name == "is_valid_timestamp":
                col_name = param_columnas_str.strip() if param_columnas_str else None
                if col_name: params_dict["column"] = col_name
                if param_valor: params_dict["timestamp_format"] = param_valor
            elif technical_name in ["is_in_range","is_not_in_range"]:
                col_name = param_columnas_str.strip() if param_columnas_str else None
                if col_name: params_dict["column"] = col_name
                if param_min: params_dict["min_limit"] = float(param_min)
                if param_max: params_dict["max_limit"] = float(param_max)
            elif technical_name in ["is_not_less_than"]:
                col_name = param_columnas_str.strip() if param_columnas_str else None
                if col_name: params_dict["column"] = col_name
                if param_min: params_dict["limit"] = float(param_min)
            elif technical_name in ["is_not_greater_than"]:
                col_name = param_columnas_str.strip() if param_columnas_str else None
                if col_name: params_dict["column"] = col_name
                if param_max: params_dict["limit"] = float(param_max)
            elif technical_name == "sql_expression" and param_sql:
                params_dict["expression"] = param_sql
            elif technical_name == "sql_query" and param_query:
                params_dict["query"] = param_query
                merge_cols = [c.strip() for c in param_merge.split(',') if c]
                if merge_cols: params_dict["merge_columns"] = merge_cols
        elif rule_type in ['CUSTOM']:
            cols = [c.strip() for c in param_columnas_str.split(',') if c]
            if cols: params_dict["columns"] = cols
        return json.dumps(params_dict)
    except Exception as e:
        return json.dumps({"error": str(e)})

build_json_udf = udf(build_json_definition, StringType())

# DBTITLE 4,4. Función genérica para sincronizar Delta por hoja
def sync_delta(df_spark, target_table_name, merge_keys, column_mapping, sheet_name):
    if df_spark is None:
        return 0, []
    errors_list = []
    try:
        delta_table = DeltaTable.forName(spark, target_table_name)
        delta_table.alias("target").merge(
            df_spark.alias("source"),
            " AND ".join([f"target.{k} = source.{k}" for k in merge_keys])
        ).whenMatchedUpdate(set={k:v for k,v in column_mapping.items()}) \
         .whenNotMatchedInsert(values={k:v for k,v in column_mapping.items()}) \
         .execute()
        return df_spark.count(), errors_list
    except Exception as e:
        errors_list.append({"sheet": sheet_name, "fila": None, "error": str(e)})
        return 0, errors_list

# DBTITLE 5,5. Sincronización específica por hoja
def sync_tables(df_excel):
    df_valid = df_excel.select(
        col("Id_tabla").alias("table_id"),
        col("Nombre_tabla").alias("table_name"),
        col("Nombre_técnico").alias("table_name_tech"),
        col("Clave_primaria").alias("primary_key"),
        col("Tabla_staging_evidencias").alias("staging_evidences_table")
    ).filter(col("Id_tabla") != "")
    mapping = {c:c for c in df_valid.columns}
    return sync_delta(df_valid, TABLE_CONFIG, ["table_id"], mapping, "Tablas")

def sync_rules(df_excel):
    df_valid = df_excel.select(
        col("Id_regla").alias("rule_id"),
        col("Nombre_técnico").alias("technical_rule_name"),
        col("Nombre_funcional").alias("functional_name"),
        col("Descripción").alias("description"),
        col("Dimensión_reglas").alias("dimension_dq"),
        col("Tipo_implementación").alias("implementation_type"),
        current_timestamp().alias("updated_at"),
        col("Actualizado_por").alias("updated_by"),
        split(col("Etiquetas"), ",\s*").alias("tags"),
        col("Propietario").alias("owner")
    ).filter(col("Id_regla") != "")
    mapping = {c:c for c in df_valid.columns}
    return sync_delta(df_valid, RULE_LIB_TABLE, ["rule_id"], mapping, "Reglas")

def sync_validations(df_excel):
    df_valid = df_excel.select(
        col("Id_validación").alias("validation_id"),
        col("Id_regla").alias("rule_id"),
        col("Id_tabla").alias("table_id"),
        col("Definición_perimetro").alias("perimeter_definition"),
        col("Validación_activa").cast(BooleanType()).alias("is_active"),
        col("Severidad").alias("severity"),
        build_json_udf(
            col("Nombre_técnico"), col("Tipo_regla"),
            col("Param_columnas"), col("Param_valor"),
            col("Param_valor_min"), col("Param_valor_max"),
            col("Param_conjunto"), col("Param_expresion"),
            col("Param_tipo_dato"), col("Param_query_SQL"),
            col("Param_merge_columnas")
        ).alias("validation_definition"),
        col("Dominio_tabla").alias("domain"),
        split(col("Param_columnas"), ",\s*").alias("Param_columns"),
        col("Param_valor").alias("Param_value"),
        col("Param_valor_min").alias("Param_value_min"),
        col("Param_valor_max").alias("Param_value_max"),
        col("Param_conjunto").alias("Param_range"),
        col("Param_expresion").alias("Param_expression"),
        col("Param_tipo_dato").alias("Param_data_type"),
        col("Param_query_SQL").alias("Param_query_SQL"),
        col("Param_merge_columnas").alias("Param_merge_columns"),
        col("Owner").alias("owner"),
        current_timestamp().alias("updated_at"),
        col("Actualizado_por").alias("updated_by"),
        split(col("Etiquetas"), ",\s*").alias("tags")
    ).filter(col("Id_validación") != "")
    mapping = {c:c for c in df_valid.columns}
    return sync_delta(df_valid, CATALOG_TABLE, ["validation_id"], mapping, "Validaciones")

# DBTITLE 6,6. Main Orquestador
def main():
    total_errors = []
    try:
        # 1. Tablas
        df_tablas, errors_tablas = load_excel_sheet(EXCEL_PATH, CONFIG_SHEET, CONFIG_EXPECTED_COLS)
        total_errors.extend(errors_tablas)
        synced_tablas, _ = sync_tables(df_tablas)
        
        # 2. Reglas
        df_rules, errors_rules = load_excel_sheet(EXCEL_PATH, LIBRARY_SHEET, LIBRARY_EXPECTED_COLS)
        total_errors.extend(errors_rules)
        synced_rules, _ = sync_rules(df_rules)
        
        # 3. Validaciones
        df_valids, errors_valids = load_excel_sheet(EXCEL_PATH, VALIDATIONS_SHEET, VALIDATIONS_EXPECTED_COLS)
        total_errors.extend(errors_valids)
        synced_valids, _ = sync_validations(df_valids)
        
        print(f"Sincronización completa: {synced_tablas} tablas, {synced_rules} reglas, {synced_valids} validaciones.")
        if total_errors:
            print(f"Se detectaron {len(total_errors)} errores durante la sincronización:")
            for e in total_errors: print(e)
        return f"Éxito: Tablas {synced_tablas}, Reglas {synced_rules}, Validaciones {synced_valids}, Errores {len(total_errors)}"
    except Exception as e:
        print(f"Fallo crítico: {e}")
        raise

# DBTITLE 7,7. Punto de entrada
if __name__ == "__main__":
    result_msg = main()
    dbutils.notebook.exit(result_msg)